In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
sample = pd.read_stata('../data/sample.dta')
sample.head()

,id,Ofirstnam,Omidnam,Osurname,msa
0,1.0,A,DALE,MAYO,9999.0
1,2.0,MARY,ANN,SKREZEC,7362.0
2,3.0,RAY,HOWARD,SUHLER,6450.0
3,4.0,VUONG,TUYEN,KHON,9999.0
4,5.0,SON,SUK,LEE,3920.0


## AAAI Conference

In [3]:
aaai_df = pd.read_csv('../data/aaai_authors.csv')
len(aaai_df)

# 2004
# 97
# change from 2002
# if ; in jj then:
# split by , and remove last as university

29349

In [4]:
aaai_df.head()

,year,paper_id,title,url,author_name,university,country
0,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Xiaohui Bei,Nanyang Technological University,Singapore
1,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Shengyu Zhang,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Pin-Yu Chen,IBM Research AI,United States
3,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Yash Sharma,NaN,"The Cooper Union, New York"
4,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Huan Zhang,NaN,"University of California, Davis"


## Only keep from 2018 - 2005

In [5]:
aaai_df2 = pd.read_csv('../data/aaai_18_05.csv')
total_aaai2 = len(aaai_df2)
aaai_df2.head()

,year,paper_id,title,url,author_name,university,country
0,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Xiaohui Bei,Nanyang Technological University,Singapore
1,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Shengyu Zhang,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Pin-Yu Chen,IBM Research AI,United States
3,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Yash Sharma,NaN,"The Cooper Union, New York"
4,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Huan Zhang,NaN,"University of California, Davis"


In [6]:
def clean_and_name(name):
    name = str(name)
    if name.split(' ')[0] == 'and':
        return ' '.join(name.split(' ')[1:])
    if name.split(' ')[-1] == 'and':
        return ' '.join(name.split(' ')[:-1])
    else:
        return name

def split_name(name):
    name = clean_and_name(name)
    first_name = None
    middle_name = None
    surname = None
    name_split = str(name).split(' ')
    if len(name_split) == 1:
        first_name  = name_split[0]
    elif len(name_split) == 2:
        first_name  = name_split[0]
        surname = name_split[1]
    elif len(name_split) == 3:
        first_name  = name_split[0]
        middle_name = name_split[1]
        surname = name_split[2]
    else:
        return pd.Series([None, None, None])
    if  len(re.sub('[.]', '', first_name)) == 1:
        return pd.Series([None, None, None])
    else:
        return pd.Series([first_name, middle_name, surname])

In [7]:
aaai_df2['first_name'] = None
aaai_df2['middle_name'] = None
aaai_df2['surname'] = None
aaai_df2[['first_name', 'middle_name', 'surname']] = aaai_df2['author_name'].apply(lambda x: split_name(x))
aaai_df2[['author_name', 'first_name', 'middle_name', 'surname', 'country']].head()

,author_name,first_name,middle_name,surname,country
0,Xiaohui Bei,Xiaohui,None,Bei,Singapore
1,Shengyu Zhang,Shengyu,None,Zhang,Hong Kong
2,Pin-Yu Chen,Pin-Yu,None,Chen,United States
3,Yash Sharma,Yash,None,Sharma,"The Cooper Union, New York"
4,Huan Zhang,Huan,None,Zhang,"University of California, Davis"


In [11]:
first_aaai = aaai_df2[['first_name', 'middle_name', 'surname', 'country']].copy()
first_aaai = first_aaai.drop_duplicates()
unique_aaai = len(first_aaai)
first_aaai = first_aaai[~(pd.isnull(first_aaai['first_name']) | 
                                 (first_aaai['first_name'] == 'None'))].copy()
#first_aaai = first_aaai.dropna(subset=['first_name'])
unique_clean_names = len(first_aaai)
first_aaai.reset_index(inplace=True)
first_aaai.rename(columns={'index':'id', 
                           'first_name': 'Ofirstnam', 
                           'middle_name': 'Omidnam', 
                           'surname': 'Osurname'}, inplace=True)
first_aaai['id'] = first_aaai['id'].apply(lambda x: 'aaai_1_{}'.format(x))
first_aaai.to_csv('../data/clean_name/aaai_first_18_05.csv', index=False)
print('papers {}'.format(total_aaai2))
print('unique {}'.format(unique_aaai))
print('unique clean {}'.format(unique_clean_names))
first_aaai

papers 20233
unique 14417
unique clean 14293


,id,Ofirstnam,Omidnam,Osurname,country
0,aaai_1_0,Xiaohui,None,Bei,Singapore
1,aaai_1_1,Shengyu,None,Zhang,Hong Kong
2,aaai_1_2,Pin-Yu,None,Chen,United States
3,aaai_1_3,Yash,None,Sharma,"The Cooper Union, New York"
4,aaai_1_4,Huan,None,Zhang,"University of California, Davis"
5,aaai_1_5,Jinfeng,None,Yi,Tencent AI Lab
6,aaai_1_6,Cho-Jui,None,Hsieh,"University of California, Davis"
7,aaai_1_7,Jonathan,None,Chung,University of Toronto
8,aaai_1_8,Moshe,None,Eizenman,University of Toronto
9,aaai_1_9,Uros,None,Rakita,University of Toronto


In [9]:
second_aaai = aaai_df2[pd.isnull(aaai_df2['first_name'])]
second_aaai

,year,paper_id,title,url,author_name,university,country,first_name,middle_name,surname
75,18,17255,On Organizing Online Soirees with Live Multi-S...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,C. P. Kankeu Fotsing,NaN,Academia Sinica,None,None,None
318,18,17389,"Perceiving, Learning, and Recognizing 3D Obj...",https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,S. Hamidreza Kasaei,University of Aveiro,Portugal,None,None,None
453,18,17346,Asymmetric Action Abstractions for Multi-Unit ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Levi H. S. Lelis,NaN,Universidade Federal de Viçosa,None,None,None
523,18,16812,Facility Location Games with Fractional Prefer...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Chi Kit Ken Fong,NaN,City University of Hong Kong,None,None,None
777,18,16926,Deception Detection in Videos,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,V. S. Subrahmanian,NaN,Dartmouth College,None,None,None
855,18,16413,Behavior Is Everything: Towards Representing C...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,D. Scott Phoenix,NaN,Vicarious AI,None,None,None
930,18,16590,On the Satisfiability Problem of Patterns in S...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Jan Van den Bussche,NaN,Hasselt University,None,None,None
939,18,16157,Deep-Treat: Learning Optimal Personalized Trea...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Mihaela van der Schaar,NaN,University of Oxford; University of California...,None,None,None
1057,18,16160,DeepHit: A Deep Learning Approach to Survival ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Mihaela van der Schaar,NaN,University of Oxford; University of California...,None,None,None
1069,18,17037,Unified Locally Linear Classifiers with Divers...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Steven C. H. Hoi,NaN,Singapore Management University,None,None,None
